In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
mnist_dataset, mnist_info = tfds.load(name='mnist', as_supervised=True, with_info=True)

In [3]:
mnist_dataset

{'train': <PrefetchDataset shapes: ((28, 28, 1), ()), types: (tf.uint8, tf.int64)>,
 'test': <PrefetchDataset shapes: ((28, 28, 1), ()), types: (tf.uint8, tf.int64)>}

In [4]:
mnist_info

tfds.core.DatasetInfo(
    name='mnist',
    full_name='mnist/3.0.1',
    description="""
    The MNIST database of handwritten digits.
    """,
    homepage='http://yann.lecun.com/exdb/mnist/',
    data_path='C:\\Users\\jangd\\tensorflow_datasets\\mnist\\3.0.1',
    download_size=11.06 MiB,
    dataset_size=21.00 MiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'train': <SplitInfo num_examples=60000, num_shards=1>,
    },
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann.lecun.com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
)

In [5]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']


In [6]:
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples

In [7]:
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

In [8]:
num_test_samples = mnist_info.splits['test'].num_examples

In [9]:
num_test_samples = tf.cast(num_test_samples, tf.int64)

In [10]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label


In [11]:
scaled_train_and_validation_data = mnist_train.map(scale)

In [12]:
test_data = mnist_test.map(scale)

In [13]:
BUFFER_SIZE = 10000

In [14]:
shuffled_trained_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

In [15]:
validation_data = shuffled_trained_and_validation_data.take(num_validation_samples)

In [16]:
train_data = shuffled_trained_and_validation_data.skip(num_validation_samples)

In [17]:
BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)

In [18]:
validation_data = validation_data.batch(num_validation_samples)

In [19]:
test_data = test_data.batch(num_test_samples)

In [20]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(units=50, activation='relu'),
    tf.keras.layers.Dense(units=50, activation='relu'),
    tf.keras.layers.Dense(units=10, activation='softmax')
])

In [21]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [22]:
validation_inputs, validation_targets = next(iter(validation_data))

In [23]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

In [24]:
model.fit(train_data, epochs=50, validation_data=(validation_inputs, validation_targets), verbose=2, callbacks=[early_stopping])

Epoch 1/50
540/540 - 9s - loss: 0.4091 - accuracy: 0.8838 - val_loss: 0.2117 - val_accuracy: 0.9372
Epoch 2/50
540/540 - 5s - loss: 0.1857 - accuracy: 0.9453 - val_loss: 0.1526 - val_accuracy: 0.9542
Epoch 3/50
540/540 - 4s - loss: 0.1421 - accuracy: 0.9584 - val_loss: 0.1239 - val_accuracy: 0.9637
Epoch 4/50
540/540 - 5s - loss: 0.1152 - accuracy: 0.9661 - val_loss: 0.1065 - val_accuracy: 0.9703
Epoch 5/50
540/540 - 5s - loss: 0.1005 - accuracy: 0.9700 - val_loss: 0.1003 - val_accuracy: 0.9715
Epoch 6/50
540/540 - 4s - loss: 0.0864 - accuracy: 0.9736 - val_loss: 0.0935 - val_accuracy: 0.9730
Epoch 7/50
540/540 - 4s - loss: 0.0764 - accuracy: 0.9768 - val_loss: 0.0895 - val_accuracy: 0.9725
Epoch 8/50
540/540 - 4s - loss: 0.0674 - accuracy: 0.9796 - val_loss: 0.0921 - val_accuracy: 0.9727
Epoch 9/50
540/540 - 5s - loss: 0.0627 - accuracy: 0.9806 - val_loss: 0.0740 - val_accuracy: 0.9778
Epoch 10/50
540/540 - 5s - loss: 0.0551 - accuracy: 0.9834 - val_loss: 0.0799 - val_accuracy: 0.9753

In [26]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 1s 1s/step - loss: 0.1220 - accuracy: 0.9721


In [27]:
test_loss

0.12196633219718933

In [28]:
test_accuracy

0.972100019454956